In [ ]:
# Paling Efektif
import cv2, numpy as np
from sklearn.cluster import KMeans

cap = cv2.VideoCapture(0)
cv2.namedWindow("ori-blur-bin", cv2.WINDOW_NORMAL)
cv2.resizeWindow("ori-blur-bin", (1280, 360))
cv2.createTrackbar("Kernel Size", "ori-blur-bin", 1, 31, lambda x: None)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    original_img = frame.copy()
    k_val = cv2.getTrackbarPos("Kernel Size", "ori-blur-bin")
    if k_val % 2 == 0:
        k_val += 1
    if k_val <= 0:
        k_val = 1
    blurred_img = cv2.GaussianBlur(frame, (k_val, k_val), 5)
    h, w = blurred_img.shape[:2]
    data = blurred_img.reshape(-1, 3).astype(np.float32)

    km = KMeans(n_clusters=2, random_state=42).fit(data)
    labels = km.labels_.reshape(h, w)
    centers = km.cluster_centers_

    bright = []
    for c in centers:
        b, g, r = c  # format BGR
        bright.append(0.114 * b + 0.587 * g + 0.299 * r)
    # Cluster dengan kecerahan terendah dianggap garis
    line_cluster = np.argmin(bright)

    # Pixel pada cluster garis = hitam (0), selainnya putih (255)
    mask = np.where(labels == line_cluster, 0, 255).astype(np.uint8)

    cv2.imshow("ori-blur-bin", np.hstack([original_img, blurred_img, cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)]))
    if cv2.waitKey(1) & 0xFF == ord("n"):
        break
cap.release()
cv2.destroyAllWindows()